ระบบแนะนำภาพยนต์

In [70]:
# data = df.rename(columns={"Animation|Children's|Comedy":'topics'})

import pandas as pd 
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv('movies.csv').dropna()
df

,MovieID,Title,MPAA Rating,Budget,Gross,Release Date,Genre,Runtime,Rating,Rating Count,Summary
0,1.0,Look Who's Talking,PG-13,7500000.0,296000000.0,1989-10-12,Romance,93.0,5.9,73638.0,"After a single, career-minded woman is left on..."
1,2.0,Driving Miss Daisy,PG,7500000.0,145793296.0,1989-12-13,Comedy,99.0,7.4,91075.0,An old Jewish woman and her African-American c...
2,3.0,Turner & Hooch,PG,13000000.0,71079915.0,1989-07-28,Crime,100.0,7.2,91415.0,"Det. Scott Turner (Tom Hanks) is an uptight, b..."
3,4.0,Born on the Fourth of July,R,14000000.0,161001698.0,1989-12-20,War,145.0,7.2,91415.0,The biography of Ron Kovic. Paralyzed in the V...
4,5.0,Field of Dreams,PG,15000000.0,84431625.0,1989-04-21,Drama,107.0,7.5,101702.0,"An Iowa corn farmer, hearing voices, interpret..."
...,...,...,...,...,...,...,...,...,...,...,...
505,506.0,Big Hero 6,PG,165000000.0,652105443.0,2014-10-24,Animation,102.0,7.8,380953.0,The special bond that develops between plus-si...
506,507.0,Interstellar,PG-13,165000000.0,675120017.0,2014-11-05,Science Fiction,169.0,8.6,1343549.0,A team of explorers travel through a wormhole ...
507,508.0,Captain America: The Winter Soldier,PG-13,170000000.0,714766572.0,2014-03-20,Action,136.0,7.7,685903.0,As Steve Rogers struggles to embrace his role ...
508,509.0,Dawn of the Planet of the Apes,PG-13,170000000.0,710644566.0,2014-06-26,Science Fiction,130.0,7.6,395425.0,A growing nation of genetically evolved apes l...


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 496 entries, 0 to 509
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   MovieID       496 non-null    float64
 1   Title         496 non-null    object 
 2   MPAA Rating   496 non-null    object 
 3   Budget        496 non-null    float64
 4   Gross         496 non-null    float64
 5   Release Date  496 non-null    object 
 6   Genre         496 non-null    object 
 7   Runtime       496 non-null    float64
 8   Rating        496 non-null    float64
 9   Rating Count  496 non-null    float64
 10  Summary       496 non-null    object 
dtypes: float64(6), object(5)
memory usage: 46.5+ KB


In [63]:
#ดูข้อมูลที่สูญหาย (Missing data)
df.isnull().sum()

MovieID         0
Title           0
MPAA Rating     0
Budget          0
Gross           0
Release Date    0
Genre           0
Runtime         0
Rating          0
Rating Count    0
Summary         0
dtype: int64

In [64]:
#ค้นหาชื่อภาพยนต์
df[df.Genre.str.contains('Comedy', case='False')].Genre

1      Comedy
8      Comedy
9      Comedy
10     Comedy
14     Comedy
        ...  
456    Comedy
476    Comedy
479    Comedy
495    Comedy
496    Comedy
Name: Genre, Length: 92, dtype: object

In [72]:
#ต้องการดูว่าภาพยนต์แถวที่ 456 มีอะไรบ้าง 
df.iloc[456]

MovieID                                                     468.0
Title                                 Snow White and the Huntsman
MPAA Rating                                                 PG-13
Budget                                                170000000.0
Gross                                                 396600000.0
Release Date                                           2012-05-30
Genre                                                     Fantasy
Runtime                                                     127.0
Rating                                                        6.1
Rating Count                                             262152.0
Summary         In a twist to the fairy tale, the Huntsman ord...
Name: 467, dtype: object

In [73]:
#ดูรายละเอียดหมดทุกคอลัมน์

for k in df.columns:
    print(k, ':', df[k].iloc[456])
    print('____________')

MovieID : 468.0
____________
Title : Snow White and the Huntsman
____________
MPAA Rating : PG-13
____________
Budget : 170000000.0
____________
Gross : 396600000.0
____________
Release Date : 2012-05-30
____________
Genre : Fantasy
____________
Runtime : 127.0
____________
Rating : 6.1
____________
Rating Count : 262152.0
____________
Summary : In a twist to the fairy tale, the Huntsman ordered to take Snow White into the woods to be killed winds up becoming her protector and mentor in a quest to vanquish the Evil Queen.
____________


In [80]:
df.columns

Index(['MovieID', 'Title', 'MPAA Rating', 'Budget', 'Gross', 'Release Date',
       'Genre', 'Runtime', 'Rating', 'Rating Count', 'Summary'],
      dtype='object')

In [81]:
features = ['Title','Genre','Summary']
df[features].isnull().sum()

Title      0
Genre      0
Summary    0
dtype: int64

ทำการรวมคอลัมน์ ['Title','Genre','Summary'] ให้เป็นคอลัมน์เดียว

In [82]:
df['combined_features'] = df.Title + '' + df.Genre + '' + df.Summary

In [85]:
df.combined_features.head()

0    Look Who's TalkingRomanceAfter a single, caree...
1    Driving Miss DaisyComedyAn old Jewish woman an...
2    Turner & HoochCrimeDet. Scott Turner (Tom Hank...
3    Born on the Fourth of JulyWarThe biography of ...
4    Field of DreamsDramaAn Iowa corn farmer, heari...
Name: combined_features, dtype: object

In [86]:
df.iloc[0].combined_features
df.combined_features[0]

"Look Who's TalkingRomanceAfter a single, career-minded woman is left on her own to give birth to the child of a married man, she finds a new romantic chance in a cab driver. Meanwhile, the point-of-view of the newborn boy is narrated through voice-over."

การประมวลผล คือการคำนวณหาค่าความคล้าย 

In [103]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#ตรวจสอบขนาดของ matrix 
cv = CountVectorizer()
count_matrix = cv.fit_transform(df.combined_features)
count_matrix.shape

(496, 4362)

In [104]:
#ตรวจสอบตารางค่า 10 ตัวแรก และ 10 ตัวสุดท้าย
print(cv.get_feature_names()[0:10])
print(cv.get_feature_names()[-10:])

['00', '000', '007', '101', '11', '11dramamichael', '12', '13', '13dramanasa', '1885']
['yuppie', 'zefram', 'zeus', 'zion', 'zombie', 'zone', 'zoo', 'zorro', 'zorroactiona', 'zscience']


C:\Users\Techatach\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [105]:
#แสดงตารางความถี่ 
print(count_matrix.toarray()[:10, 10:30])

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


คำนวณหาค่าความคล้าย Cosine Similarity

In [106]:
cosine_sim = cosine_similarity(count_matrix)
cosine_sim.shape

(496, 496)

In [107]:
cosine_sim

array([[1.        , 0.23769134, 0.36079009, ..., 0.27897624, 0.44067797,
        0.40087788],
       [0.23769134, 1.        , 0.16865481, ..., 0.23473824, 0.14261481,
        0.14054567],
       [0.36079009, 0.16865481, 1.        , ..., 0.31341872, 0.31569133,
        0.33333333],
       ...,
       [0.27897624, 0.23473824, 0.31341872, ..., 1.        , 0.16738574,
        0.30242157],
       [0.44067797, 0.14261481, 0.31569133, ..., 0.16738574, 1.        ,
        0.40087788],
       [0.40087788, 0.14054567, 0.33333333, ..., 0.30242157, 0.40087788,
        1.        ]])

In [112]:
#ดูความคล้ายกันในแต่ละแถว
print(cosine_sim[:8, :8].round(3))

[[1.    0.238 0.361 0.323 0.221 0.026 0.071 0.17 ]
 [0.238 1.    0.169 0.24  0.173 0.251 0.267 0.08 ]
 [0.361 0.169 1.    0.269 0.175 0.023 0.105 0.151]
 [0.323 0.24  0.269 1.    0.248 0.086 0.16  0.095]
 [0.221 0.173 0.175 0.248 1.    0.148 0.104 0.165]
 [0.026 0.251 0.023 0.086 0.148 1.    0.215 0.171]
 [0.071 0.267 0.105 0.16  0.104 0.215 1.    0.08 ]
 [0.17  0.08  0.151 0.095 0.165 0.171 0.08  1.   ]]
